In [ ]:
import sys, time, os, asyncio, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
client = EfdClient('usdf_efd')

# First, get the data

In [ ]:
startDay1 = 20241105
endDay1 = 20241107
startString1 = dayObsIntToString(startDay1)
endString1 = dayObsIntToString(endDay1)
start1 = Time(f"{startString1}T00:00:00", scale='utc')
end1 = Time(f"{endString1}T04:00:00", scale='utc')
startDay2 = 20241123
endDay2 = 20241125
startString2 = dayObsIntToString(startDay2)
endString2 = dayObsIntToString(endDay2)
start2 = Time(f"{startString2}T00:00:00", scale='utc')
end2 = Time(f"{endString2}T04:00:00", scale='utc')


In [ ]:
m2Temp1 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem3', 'sensorName'],  start1, end1, index=201)
m2Temp1 = m2Temp1[m2Temp1['sensorName'] == 'AuxTel-ESS01']   
m2Temp1 = m2Temp1.shift(18, freq='D')
m1Temp1 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem5', 'sensorName'],  start1, end1, index=201)
m1Temp1 = m1Temp1[m1Temp1['sensorName'] == 'AuxTel-ESS03']    
m1Temp1 = m1Temp1.shift(18, freq='D')
outTemp1 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem0'],  start1, end1, index=301)
outTemp1 = outTemp1.shift(18, freq='D')
m2Temp2 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem3', 'sensorName'],  start2, end2, index=201)
m2Temp2 = m2Temp2[m2Temp2['sensorName'] == 'AuxTel-ESS01'] 

m1Temp2 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem5', 'sensorName'],  start2, end2, index=201)
m1Temp2 = m1Temp2[m1Temp2['sensorName'] == 'AuxTel-ESS03']                                        

outTemp2 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem0'],  start2, end2, index=301)


# Now plot it.

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.subplots_adjust(wspace=0.3)
plt.suptitle("Impact of AuxTel painting on in-dome temperatures")
axs[0].set_title("Air Temperatures")
m2Temp1['temperatureItem3'].plot(ax=axs[0], color='blue', label = "DomeT(shifted), no paint")
m2Temp2['temperatureItem3'].plot(ax=axs[0], color='red', label = "DomeT, paint")
outTemp1['temperatureItem0'].plot(ax=axs[0], color='cyan', label = "Outside(shifted), no paint")
outTemp2['temperatureItem0'].plot(ax=axs[0], color='magenta', label = "Outside, paint")
axs[0].legend(loc='upper left')
axs[0].set_ylim(10,35)
axs[0].set_ylabel("Temp (C)")

axs[1].set_title("M1 mirror Temperatures")
m1Temp1['temperatureItem5'].plot(ax=axs[1], color='blue', label = "M1 temp(shifted), no paint")
m1Temp2['temperatureItem5'].plot(ax=axs[1], color='red', label = "M1 temp, paint")
#outTemp1['temperatureItem0'].plot(ax=axs[1], color='cyan', label = "Outside, no paint")
#outTemp2['temperatureItem0'].plot(ax=axs[1], color='magenta', label = "Outside(shifted), paint")
axs[1].legend(loc='upper left')
axs[1].set_ylim(10,35)
axs[1].set_ylabel("Temp (C)")
plt.savefig("/home/c/cslage/u/AuxTel/dome/Painting_Impact_26Nov24.png")